# #Machine learning Model 

In [ ]:
import cv2
import time
from keras.models import load_model
import numpy as np
from PIL import Image
import pandas as pd
from datetime import datetime
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtGui import QImage, QPixmap

#### Initialize face detector and model

In [1]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
model = load_model('Age_Gender_Detection.keras')

### Data-frame to store senior citizens data
data = pd.DataFrame(columns=['Age', 'Gender', 'Time of Visit'])

#### Function to predict the age and the gender
def predict_age_gender(face):
    face_img = Image.fromarray(face).resize((48, 48))
    face_img = np.array(face_img)
    face_img = np.expand_dims(face_img, axis=0) / 255.0

    ### Predicting the age and the gender 
    pred = model.predict(face_img)
    age = int(np.round(pred[1][0]))
    gender = "Male" if np.round(pred[0][0]) == 1 else "Female"

    return age, gender

###Function to save details of senior citizens data to csv file 
def save_to_csv(age, gender):
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    global data
    data = data.append({'Age': age, 'Gender': gender, 'Time of Visit': current_time}, ignore_index=True)
    data.to_csv('senior_citizens_log.csv', index=False)

# Function to check if the person is a senior citizen
def check_senior(age, gender):
    if age > 60:
        print(f"Senior Citizen Detected: {age}, {gender}")
        save_to_csv(age, gender)


class AgeGenderDetectorApp(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("AGE & GENDER DETECTOR")
        self.setWindowFlag(Qt.FramelessWindowHint)  
        self.showFullScreen()

        ### creating layout to hold video feed 
        self.layout = QVBoxLayout()
        self.video_label = QLabel()
        self.layout.addWidget(self.video_label)
        self.setLayout(self.layout)

        self.cap = cv2.VideoCapture(0)
         # Interval of 30ms for video refresh
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(30)  

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            face_roi = frame[y:y+h, x:x+w]

            age, gender = predict_age_gender(face_roi)

            ### save the data if their is senior citizen
            check_senior(age, gender)

            ### Displaying the age and gender
            cv2.putText(frame, f"Age: {age}", (x, y-40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.putText(frame, f"Gender: {gender}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        #### Convert frame to QImage for display in PyQt5
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        height, width, channel = rgb_frame.shape
        bytes_per_line = 3 * width
        q_img = QImage(rgb_frame.data, width, height, bytes_per_line, QImage.Format_RGB888)

        self.video_label.setPixmap(QPixmap.fromImage(q_img))

    def closeEvent(self, event):
        #### Release video capture when the window is closed 
        self.cap.release()
        cv2.destroyAllWindows()


if __name__ == '__main__':
    app = QApplication([])

    #### Starting the Application 
    window = AgeGenderDetectorApp()
    window.show()
    app.exec_()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

C:\Users\jhara\AppData\Local\Temp\ipykernel_6504\332770116.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  age = int(np.round(pred[1][0]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━